# Problem Statement
Employees can belong to multiple departments. When employees join other departments, they need to decide which department is their primary one. Note that when an employee belongs to only one department, their primary column is 'N'.

Write a solution to report all the employees with their primary department. For employees who belong to one department, report their only department.

Return the result table in any order.

**Example 1:**

**Input:** 
Employee table:
| employee_id | department_id | primary_flag |
|-------------|---------------|--------------|
| 1           | 1             | N            |
| 2           | 1             | Y            |
| 2           | 2             | N            |
| 3           | 3             | N            |
| 4           | 2             | N            |
| 4           | 3             | Y            |
| 4           | 4             | N            |

**Output:** 
| employee_id | department_id |
|-------------|---------------|
| 1           | 1             |
| 2           | 1             |
| 3           | 3             |
| 4           | 3             |

# Intuition
We need to identify the primary department for employees with multiple departments and, for those with only one department, report that department. This involves querying for employees with a primary flag of 'Y' for those with multiple departments and selecting the single department for others.

This is the sort of problem where I actually learned a few things. I couldn't figure out by myself the most efficient solution (numero 2), figured out solution 3 but it turns out that one was not the best way to proceed. Saw other solutions provided by Leetcoders and LLMs --> the final veridict is indeed solution 2. See below.

# Approach

## Solution 1: Using `UNION`

```sql
SELECT employee_id, department_id
FROM Employee 
WHERE primary_flag = 'Y'
UNION
SELECT employee_id, department_id
FROM Employee 
GROUP BY employee_id
HAVING COUNT(employee_id) = 1
ORDER BY employee_id;

```sql
```
#### Pros: 
Simple and clear separation of logic for multi vs. single-department employees.
Works well without needing complex SQL constructs like window functions.

#### Cons: 
Requires scanning the table twice, which might impact performance for large datasets.

When to Use: 
When you need straightforward SQL, especially in environments where performance isn't critical or the dataset is small to medium.

# Solution 2: Using Window Functions

```sql
SELECT 
  employee_id, 
  department_id 
FROM 
  (
    SELECT 
      *, 
      COUNT(employee_id) OVER(PARTITION BY employee_id) AS EmployeeCount
    FROM 
      Employee
  ) EmployeePartition 
WHERE 
  EmployeeCount = 1 
  OR primary_flag = 'Y';

```sql

```
#### Pros:
Scans the table only once, which can be more efficient for larger datasets.
Uses window functions to count departments per employee directly in the query.

#### Cons:
Window functions might be computationally expensive or less optimized in some database systems.
Uses a subquery which might add some overhead.

When to Use:
Prefer when dealing with larger datasets where minimizing table scans is beneficial, or if your SQL system has well-optimized window functions.


# Solution 3: Using WITH CTE + JOIN
```sql

WITH n_dept AS (
    SELECT employee_id, department_id, COUNT(department_id) AS ndept
    FROM Employee
    GROUP BY employee_id
)
SELECT e.employee_id, e.department_id
FROM Employee e
JOIN n_dept nd ON nd.employee_id = e.employee_id
WHERE e.primary_flag = "Y" OR nd.ndept = 1;

```sql
```
#### ✅ Pros:

Readable: The CTE (WITH n_dept AS (...)) makes it modular and easier to debug.
Works on databases without window functions.

#### ❌ Cons:

Requires an extra join (JOIN n_dept nd ON nd.employee_id = e.employee_id).
Less efficient than Solution 2 because:
The CTE does a full GROUP BY scan.
Then it joins back to the Employee table.
Effectively, it processes the table twice, increasing cost.


# Final Verdict:
🏆 Solution 2 (Window Function) is still the best choice for performance because:

It avoids an extra join.
It scans the table only once.
However, if an old database doesn't support window functions well, Solution 3 (WITH CTE + JOIN) is a good alternative.